In [ ]:
# DON'T RERUN THE SAVE FOR VECTOR STORE IF IT ALREADY EXISTS

In [5]:
# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# %pip install -qU "langchain[openai]"
# %pip install -qU langchain-openai
# %pip install -qU langchain-core
# %pip install langchain-chroma
%pip install pypdf



Note: you may need to restart the kernel to use updated packages.


In [21]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

docs = []
pdf_dir = Path("./bills")   # your folder with PDFs

for pdf_file in pdf_dir.glob("*.pdf"):
    loader = PyPDFLoader(str(pdf_file))
    print(str(pdf_file))
    docs.extend(loader.load())


bills/20230SB976_91.pdf
bills/congress.gov_115_plaws_publ395_PLAW-115publ395.htm.pdf
bills/CELEX_32022R2065_EN_TXT.pdf
bills/florida.pdf


In [22]:
# chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [28]:
# azure open AI key
import os
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")

In [ ]:
# embed/store
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDINGS_API_VERSION"],
)

vectordb = Chroma.from_documents(
    chunks,
    embedding=embeddings,
    persist_directory="policy_vector_store"
)
vectordb.persist()


In [30]:
# Which embeddings are you using?
print(type(embeddings))

# Does a simple embed call work?
embeddings.embed_query("hello")

<class 'langchain_openai.embeddings.azure.AzureOpenAIEmbeddings'>


[0.016751619055867195,
 -0.055799614638090134,
 0.005647437181323767,
 0.06618502736091614,
 0.008984559215605259,
 -0.04729178547859192,
 -0.02814917452633381,
 0.06102165952324867,
 -0.002181964460760355,
 -0.043741967529058456,
 0.009828007780015469,
 -0.03326853737235069,
 -0.012226334773004055,
 -0.028926612809300423,
 0.014617327600717545,
 0.05826395004987717,
 -0.07005755603313446,
 0.03755179047584534,
 0.010092043317854404,
 0.04077889770269394,
 0.06647840142250061,
 0.010121380910277367,
 -0.005618100054562092,
 0.02368989959359169,
 0.02175363525748253,
 0.011874286457896233,
 -0.004730645567178726,
 0.01930396631360054,
 0.024349989369511604,
 -0.0648941844701767,
 0.03326853737235069,
 -0.0445634126663208,
 0.020374780520796776,
 -0.00495800981298089,
 0.010884151794016361,
 -0.014793351292610168,
 -0.026271583512425423,
 0.03467673063278198,
 0.0023341518826782703,
 -0.03329787775874138,
 -0.02212035097181797,
 -0.008544499054551125,
 0.05401003733277321,
 0.02439399436